## Setup
#### Load the API key and relevant Python libaries.

In [1]:
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
from openai import OpenAI
import os
from copy import deepcopy
import json
import time
import ast
import re

import pprint

# LLM Open AI

In [6]:
# Get the first key from the uploaded dictionary
env_file_key = "../../auixiliary/env_HumanRights"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
# openai.api_key = api_key

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

# Models

In [7]:
def chat_gpt(prompt, temperature=0):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=prompt,
        temperature=temperature 
    )
    return response.choices[0].message.content.strip()

# Iterate and Save Use Riskiness Results

# Functions

In [8]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [9]:
def read_prompt_output(file_path):

  # Read the uploaded file
  with open(file_path, 'r') as file:
      data = json.load(file)

  return data

# Read Risk Reports

In [10]:
risks_categorized = read_prompt_output("../../data/FULL_RISK_REPORT_V1_138.json")

# 1 FIX PROMPT

In [12]:
MESSAGES = [
    {
        'role': 'system',
        'content': """You are an experienced regulatory compliance specialists who works in the field of AI technology regulation. You are thoughtful, decisive, experienced and conscientious.
        You have access to the entirety of the EU AI Act and its amendments, which outline how various AI technologies are to be regulated and risk-classified within the European Union."""
    },
    {
        'role': 'user',
        'content': """You will be provided in input with an AI technology use description through five concepts:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}"

        along with the risk classification according to the EU AI Act and corresponding reasoning:

        "Risk Assessment":
        "Classification": "{}",
        "Reasoning": "{}",
        "Relevant EU AI text": "{}"

        Rewrite the reasoning to be concise, fitting into one sentence of a maximum of 25 words and distilling the key info, which helps to understand why the use is high risk or low risk. The format for the rewritten statement is as follows:
        if the use is classified as "High Risk":
            "High Risk" due to [THE REASON], which falls under the EU AI Act [RELEVANT SECTION or RULE].
        or, if the classification is "Not Classified as High Risk or Unacceptable Risk":
            "Low Risk" due to [THE REASON] [mention the EU AI Act if useful only].

         Please return the rewritten "Reasoning" in the following format:
         {{
          "Reasoning": [CLASSIFICATION] due to [THE REASON], mention the EU AI Act if [RELEVANT SECTION or RULE].
        }}

        ***Check your planned output before outputting it: if it contains any explanations besides the JSON string, omit the explanations. Make sure to output ONLY a correctly formatted JSON string and nothing else.***
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject,classification,reasoning,euaiacttext):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject,classification,reasoning,euaiacttext)
    return messages


## 1 APPLY / RUN

In [13]:
# risks_categorized

In [16]:
# cost = 0

FULL_RES = []

start_time = time.time()
i = 0

found = 0

for useElements in risks_categorized:
  # print (useElements)
  useI = str(useElements['id'])

  # if int(useI) == 100:
  #   found = 1

  # if not found:
  #   continue

  print (f" Parsing use {useI}")

  # Variables for message placeholders
  domain = useElements['Details'][0]
  purpose = useElements['Details'][1]
  aiCapability = useElements['Details'][2]
  aiUser = useElements['Details'][3]
  aiSubject = useElements['Details'][4]

  classification = useElements['Classification']
  reasoning = useElements['Reasoning']
  AIActText = useElements['AIActText']

  # Extracting "Use i" details
  use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject]

  # print(use_i_details)

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, domain, purpose, aiCapability, aiUser, aiSubject, classification, reasoning, AIActText)

  # run the prompt
  response = chat_gpt(messages, temperature=0)
  print(response)

  # response, token_count = chat_gpt(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  # print (response)
  response = ast.literal_eval(response)

  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  combined_response['Classification'] = classification
  combined_response['AIActText'] = AIActText
  combined_response["Reasoning"] = response["Reasoning"]

  print (combined_response)

  end_time = time.time()

  print(f"Execution time: {end_time - start_time:.5f} seconds")
  # print (f"TOTAL COST {cost}")

  FULL_RES.append(combined_response)

 Parsing use 1
{
  "Reasoning": "High Risk due to biometric identification for access control, under EU AI Act Article 6(2) and Amendment 711."
}
{'id': '1', 'Details': ['Biometric identification and categorization of natural persons', 'Secure access control', 'Verifying identity through facial features', 'Security firms, corporations', 'Employees, visitors'], 'Classification': ['High Risk'], 'AIActText': 'High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas: 1. Biometric identification and categorisation of natural persons: (a) AI systems intended to be used for the ‘real-time’ and ‘post’ remote biometric identification of natural persons; - Amendment 711: (a) AI systems intended to be used for biometric identification of natural persons, with the exception of those mentioned in Article 5;', 'Reasoning': 'High Risk due to biometric identification for access control, under EU AI Act Article 6(2) and Amendment 711.'}
Execution time: 1.352

In [17]:
###############################
# save result
with open(f"../../results/risk_categories/FULL_Risk_Reasoning.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine

# THE END